**Scenario Narrative: Compromised Privileged Account in a Small Organization**

The setting is a small, resource-constrained organization with minimal endpoint security segmentation. The IT administrator, Steve, manages the host and has full administrative access. Due to weak internal security controls and credential reuse, the Steve account has been silently compromised—either through phishing, a leaked password, or an earlier escalation by a malicious insider.

The attacker now operates directly from this privileged context. Without the need to bypass traditional access controls, they begin making subtle but meaningful changes to the system. Their goal is to persist on the host, explore its network environment, and operate quietly without triggering immediate alarms.

This simulation models the behavior of that attacker by executing a series of host-based actions. The activity is intentionally designed to generate a clear and structured audit trail. Key attacker behaviors—such as failed logon attempts, obfuscated PowerShell execution, scheduled task creation, and simulated lateral movement—will be captured and analyzed.

The resulting telemetry will be investigated through three core tools:

Splunk SIEM for log correlation and detection

Wireshark for low-level network packet inspection during lateral movement simulation

Nmap (optional) for post-breach network surface scanning and service enumeration

The outcome is a realistic, triage-ready dataset for learning detection and investigation workflows across host, log, and network layers using real-world analyst tools.





**Insider Threat Simulation – Full Project Outline**

**Objective**
Simulate a basic insider attack using the Steve account on a Windows 11 system

Generate a telemetry trail observable via:

Windows Event Logs

Splunk SIEM

Wireshark (network visibility)

Emulate attacker behaviors including:

Failed/successful logons

Obfuscated PowerShell execution

Persistence attempt via scheduled task

Simulated lateral movement

(Optional) Privilege escalation attempt

**Scenario Narrative**

Small organization with minimal endpoint defenses

Steve (a privileged local admin) has had their account compromised

Attacker uses native tools to avoid detection and establish persistence

Simulation models this attacker behavior on a single host

**Project Outline**

1. Project Setup

**System: Windows 11**

User account: Steve (local administrator)

SIEM: Splunk (logs ingested from Windows Security log)

Logging Configuration (All confirmed enabled):

4625 – Failed Logon

4624 – Successful Logon

4688 – Process Creation

4698 – Scheduled Task Created

PowerShell Logging:

Module Logging – Enabled

Script Block Logging – Enabled

Registry auditing (Event ID 4657) – Not enabled or required

PowerShell Logging Setup (already applied):

**Command 1: Enable Module Logging**

Set-ItemProperty -Path "HKLM:\Software\Policies\Microsoft\Windows\PowerShell\ModuleLogging" -Name "EnableModuleLogging" -Value 1

**Command 2: Enable Script Block Logging**

Set-ItemProperty -Path "HKLM:\Software\Policies\Microsoft\Windows\PowerShell\ScriptBlockLogging" -Name "EnableScriptBlockLogging" -Value 1

**Wireshark Plan:**

Will be launched manually before simulated lateral movement (Step 2D)

Capture will use main network interface (Ethernet or Wi-Fi)

Optional filter: ip.addr == 192.168.56.102

**2. Simulated Intrusion**
**2A. Failed and Successful Logons**

Attempt 2–3 failed logons with incorrect password

Successfully log in as Steve

Expect Event IDs:

4625 (Failed logon)

4624 (Successful logon)

**2B. Suspicious PowerShell Execution**

Run in PowerShell:

$c = 'calc'; iex $c

Expect Event ID 4688 (calc.exe via PowerShell)

If script block logging is enabled, expect record of iex execution

**2C. Persistence via Scheduled Task**

Run:

Start-Process schtasks -ArgumentList '/create', '/tn', 'Updater', '/tr', 'calc.exe', '/sc', 'onlogon'

Expect Event ID 4698 (Scheduled Task Created)

Task will be created for the current user (no elevation needed)

**2D. Simulated Lateral Movement**

Start Wireshark

Open Wireshark on your Windows system

Select your active network interface (Ethernet or Wi-Fi)

Click “Start Capture”

Run the following two commands in PowerShell or Run dialog 

mstsc /v:192.168.56.102

Then:

net use \\192.168.56.102\C$

Wait 10–20 seconds to allow the connection attempts to complete

Stop the capture in Wireshark

Save the .pcapng file if desired

Observe in Wireshark:

TCP SYN packets outbound from your system

Dest IP: 192.168.56.102

Ports: 3389 (RDP) and 445 (SMB)



Record Wireshark start time

Record when each of the two connection commands is run

Log  Wireshark stop time

Update the timeline 

**2E. Privilege Escalation Simulation**

Attempt to run Command Prompt as Administrator

Cancel when prompted for credentials

Possible Event IDs: 4673, 4674 (Privileged service requested)

**3. Detection Signals**
This section explains how to detect the attacker’s simulated actions using host-based telemetry and network traffic captures. It also serves as a guide for interpreting the logs you’ve generated in Section 2 (Simulated Intrusion). Each entry below aligns with the attacker’s behavior and includes relevant Windows Event IDs, expected patterns in Splunk, and how Wireshark can contribute to detection visibility.

**3A. Host-Based Signals (Windows Event Logs & Splunk)**

**1. Logon Activity Pattern**

What to look for:
A cluster of failed logons (Event ID 4625) followed shortly by a successful logon (Event ID 4624) for the same account (Steve).

Why it matters:
This could indicate brute-force attempts, password spraying, or a user mistyping credentials. In a high-privilege context, it’s an early indicator of potential compromise.

How to verify in Splunk:


index=wineventlog EventCode=4625 OR EventCode=4624 user=Steve
| stats count by EventCode, Account_Name, Logon_Type, _time


**2. Obfuscated PowerShell Execution**

What to look for:
A PowerShell process (Event ID 4688) launching calc.exe via the iex (Invoke-Expression) command.
Script block logging (if enabled) should show the full obfuscated payload.

Why it matters:
Use of iex is a known red flag in attacker behavior. Combined with suspicious execution paths (like $c='calc';iex $c), this could indicate attempts to run hidden or downloaded code.

How to verify in Splunk:


index=wineventlog EventCode=4688
| search Parent_Image="*powershell.exe" New_Process_Name="*calc.exe"
To view script block logs (if collected as XML):


index=wineventlog EventCode=4104
| search Message="*iex*"

**3. Persistence via Scheduled Task**

What to look for:
Creation of a scheduled task via schtasks.exe (Event ID 4698).
The task will trigger calc.exe at logon, mimicking persistence.

Why it matters:
Adversaries commonly use scheduled tasks to survive reboots or trigger actions on specific events (like logon). Task creation by PowerShell or script context is suspicious.

How to verify in Splunk:

spl
Copy
Edit
index=wineventlog EventCode=4698
| search Task_Name="Updater"
4. Simulated Lateral Movement

What to look for:

Event ID 4648: Logon using explicit credentials for a remote share

Additional 4624 events with Logon Type 3 (network)

Why it matters:
Lateral movement attempts often generate 4648 if the attacker manually specifies a remote user/password combo. Even if login fails, the attempt may be logged.

How to verify in Splunk:

spl
Copy
Edit
index=wineventlog EventCode=4648 OR EventCode=4624 Logon_Type=3
| stats count by Account_Name, Workstation_Name, Target_Server, _time
5. Privilege Escalation Attempt

What to look for:
Attempting to open a privileged process (e.g., Command Prompt as Administrator) may trigger:

4673 (Privileged Service Called)

4674 (Operation attempted on a privileged object)

Why it matters:
Even if the escalation is blocked, the attempt itself is useful for incident triage and user behavior tracking.

How to verify in Splunk:

spl
Copy
Edit
index=wineventlog EventCode=4673 OR EventCode=4674 user=Steve
3B. Network-Based Signals (Wireshark)
1. TCP SYN Probes to Internal Hosts

What to look for in Wireshark:
Outbound SYN packets (TCP flag) from your host to IP 192.168.56.102.

Port 3389 = RDP

Port 445 = SMB

Why it matters:
These are indicators of reconnaissance or lateral movement. Even if connection fails, packet traces confirm intent.

How to verify in Wireshark:
Use the following display filter:

ini
Copy
Edit
tcp.flags.syn == 1 && ip.dst == 192.168.56.102
2. Optional Nmap Scan

If performed:
A port scan from the Steve account using Nmap would generate bursts of TCP SYN packets across many destination ports.

Detection method:
In Wireshark: look for rapid connections from your source IP to many different destination ports in sequence.

Next Steps
Once this section is executed, create a Detection Review Chart summarizing what signals were actually observed in your log and packet data:

Signal Type	Expected Artifact	Observed?	Event ID / Evidence
Failed Logons	4625	Yes/No	
Successful Logon	4624	Yes/No	
PowerShell Obfusc.	4688 + ScriptBlockLog (4104)	Yes/No	
Task Creation	4698	Yes/No	
Lateral Movement	4648 / SYN to 445, 3389	Yes/No	
Privilege Escalation	4673, 4674	Yes/No	


4. Investigation Workflow
Start with SIEM/Splunk queries:

Search for failed logons

Trace PowerShell execution

Review task creation logs

Correlate host activity timeline with Wireshark packet captures

Optionally use Nmap from another machine to probe post-attack state

Build timeline of attacker behavior using correlated sources

5. Documentation & Lessons Learned
Capture and Save:

Screenshots of log entries (Splunk, Event Viewer)

Wireshark packet captures from lateral attempt

Any Nmap scans if used

Evaluate:

Was telemetry complete?

Did Splunk surface alerts early enough?

Were PowerShell logs meaningful?

Recommendations:

Consider enhancing detection rules

Expand endpoint monitoring

Add lateral movement alerts (NetFlow, IDS) in future iterations



**2. Execution Review**
This section summarizes the actions taken during Step 2 (Simulated Intrusion) and explains their significance in the context of insider threat behavior and security telemetry generation.

**2A. Failed and Successful Logon Attempts**
Action: Three failed login attempts were made using incorrect passwords, followed by a successful login at 10:24 AM.

Purpose: Simulates unauthorized access attempts by an insider.

Log Artifacts:

4625: Failed logon attempt

4624: Successful interactive logon

Detection Value: Repeated failed logons followed by success may indicate password guessing, brute force, or credential misuse.

**2B. Suspicious PowerShell Execution**

Action: Ran PowerShell command:

$c = 'calc'; iex $c

This opened the Windows Calculator at 10:30 AM.

Purpose: Demonstrates use of Invoke-Expression (IEX), which is frequently used in obfuscated or malicious scripts.

Log Artifacts:

4688: Process creation (calc.exe via PowerShell)

Script Block Logging (if enabled): Records use of iex

Detection Value: PowerShell use of iex is often associated with malicious activity. Even benign execution (e.g., calc) serves as a recognizable signal.

**2C. Persistence via Scheduled Task**

Action: Created a scheduled task that launches calc.exe on user logon.
Successfully created at 10:45 AM after resolving permission errors.

Purpose: Simulates persistence mechanism — attacker ensures code executes on next logon.

Log Artifacts:

4698: Task creation

Task appears in Task Scheduler under name Updater

Detection Value: Unusual task creation, especially by non-admin users or tasks launching uncommon programs, is suspicious and should be reviewed.

**2D. Simulated Lateral Movement**

Action: Started Wireshark capture and ran:

mstsc /v:192.168.56.102
net use \\192.168.56.102\C$
Executed at 11:40–11:43 AM.

Purpose: Simulates attempts to pivot to another machine using RDP and SMB, even though the IP does not exist.

Artifacts:

Wireshark: Outbound SYN packets attempting RDP and SMB connection

Event Logs: May show 4624, 4648 depending on OS behavior

Detection Value: Unusual RDP/SMB attempts to nonexistent or unauthorized hosts often signal lateral movement behavior.

**2E. Privilege Escalation Attempt (Recon)**

Action: Ran the following at 11:58 AM:

vssadmin list shadows

Purpose: Attacker queries shadow copies as part of a reconnaissance step often used before password hash extraction or volume recovery.

Log Artifacts:

4688: Process creation (vssadmin.exe)

Detection Value: vssadmin usage is rare in day-to-day operations. When used by standard accounts, it's often a red flag for potential privilege escalation or post-exploitation behavior.



**Detection Signal #1: Logon Sequence Pattern**

This detection focuses on identifying a suspicious pattern of failed logon attempts followed by a successful logon. It is a foundational detection use case in insider threat or brute-force simulations, especially when the login occurs under a privileged account such as Steve in this scenario.

What We Executed
At 10:24 AM, we intentionally attempted three failed logons as the Steve user (wrong password).

We then logged in successfully on the fourth attempt.

This behavior is designed to simulate an attacker using a known or guessed username and attempting to gain access interactively.

What to Look For (Windows Event Log + Splunk)

Key Event IDs:
4625 – Failed Logon

4624 – Successful Logon

Logon Type 2 – Interactive logon (keyboard or GUI session)

Logon Type 5 – Service logon (background processes like Splunk or Windows system tasks)

Captured Sequence in Splunk:
Timestamp	Event ID	Account	Logon Type	Source Host	Notes
2025-07-01 10:24:04	4625	Steve	2	DESKTOP-UPKRV33	Failed logon #1
2025-07-01 10:24:09	4625	Steve	2	DESKTOP-UPKRV33	Failed logon #2
2025-07-01 10:24:15	4625	Steve	2	DESKTOP-UPKRV33	Failed logon #3
2025-07-01 10:24:22	4624	Steve	2	DESKTOP-UPKRV33	Successful logon
2025-07-01 10:24:23	4624	SYSTEM	5	DESKTOP-UPKRV33	System-level task
2025-07-01 10:24:33	4624	SYSTEM	5	DESKTOP-UPKRV33	System-level task
2025-07-01 10:24:49	4624	SYSTEM	5	DESKTOP-UPKRV33	System-level task

**Interpretation and Analysis**

The three 4625s represent failed login attempts using incorrect credentials.

The 4624 for Steve immediately afterward confirms successful access by the attacker or user.

Logon Type 2 means these are interactive logons — someone physically or virtually accessing the desktop.

The 4624 SYSTEM events with Logon Type 5 are normal. They reflect automated system actions triggered after a successful user login (e.g., background services, scheduled checks, Splunk processes).

The DESKTOP-UPKRV33$ entries represent the computer account's own logon behavior and trust relationship activities (also expected and benign in this context).

**Why This Matters**
In a real-world alerting system, this 4625 → 4624 pattern is often used as an early warning of:

Brute force attempts

Credential stuffing

Insider misuse

Successful pivoting after earlier access compromise

When the successful login involves a privileged account, this pattern becomes much more critical to detect.

How to Verify in Splunk
Search for this sequence using:


index=windows EventCode=4624 OR EventCode=4625
| stats count by _time, EventCode, Account_Name, Logon_Type, ComputerName
| sort _time

Then use the Advanced Time Picker to narrow the window to:


Earliest: 2025-07-01 10:24:00  
Latest:   2025-07-01 10:25:00
This sharply focuses your detection window and removes unrelated logon noise.

Additional Notes
Why not just see one 4624 after the 4625s? Because Windows logs multiple service-related logons immediately after a user logs in. This is normal, and analysts must learn to distinguish true user access from system-level background sessions.

This complexity is part of why logon telemetry is both powerful and noisy — the same Event IDs can mean very different things depending on account type, logon type, and timing.

**Supplement: Understanding DESKTOP-XXXXX$ Logon Entries**

When analyzing logon patterns in Windows Event Logs (especially Event ID 4624 for successful logons), you’ll often see accounts like:

Account Name: DESKTOP-UPKRV33$
These entries refer to the computer account itself, not a human user. This can be confusing during triage, but here’s what’s actually happening.

**What Is a Computer Account?**

In Windows, every machine that is part of a domain — or even standalone systems — has its own internal security context, represented as the machine name followed by a dollar sign ($). Example:

DESKTOP-UPKRV33$

This account is used automatically by Windows to:

Authenticate internal services

Start system processes under trusted context

Interact with local or domain resources (e.g., file shares, policies)

**Why Do You See These During a User Login?**

When a human user (e.g., Steve) logs in:

Windows doesn’t just authenticate Steve

It also performs machine-level logons behind the scenes, using DESKTOP-UPKRV33$, to initialize background services and trusted subsystems

This is especially common when:

The user has local administrator rights (like in this simulation)

The system starts services, loads profiles, or runs scheduled tasks

**Should You Investigate These?**

In most cases: No.
These are considered benign system noise unless:

The machine account is interacting with external resources abnormally

There are unexpected logon types (e.g., Type 3 over the network, or Type 10 from RDP)

The activity is happening independent of user logon times

**Best Practice for Analysts**

When performing user-focused triage:

Filter out or deprioritize machine account logons ($) unless they match suspicious patterns

Focus first on interactive user accounts and their corresponding Event IDs (4624, 4625, 4648, etc.)

This distinction helps reduce false leads and lets you zero in on attacker-driven behavior.



**Detection Signal 3A.2 – Obfuscated PowerShell Execution**
✅ Simulated Action (10:30 AM):

$c = 'calc'; iex $c

This obfuscated PowerShell line sets a variable $c to "calc" and then uses Invoke-Expression (iex) to execute it.

Although benign (it opens Calculator), this is a classic obfuscation method used by attackers to hide payloads or bypass simple command-line detection rules.

📈 Detection Signal (via Event ID 4688)
Event ID: 4688
Event Type: Process Creation
Log Source: Security Log
Time Observed: 07/01/2025 10:30:39 AM
Expected Fields:

New_Process_Name: Typically something like C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe

Process_Command_Line: Should include something like:


powershell.exe -NoProfile -ExecutionPolicy Bypass -Command "$c='calc';iex $c"

These events were observed twice, likely due to a parent-child relationship:

Parent: The PowerShell host (powershell.exe)

Child: The launched application (calc.exe)

🔍 Why It Matters
Invoke-Expression (iex) is a known red flag in threat hunting. It allows dynamic code execution, often hidden in variables, downloaded from web servers, or decrypted from payloads.

Threat actors frequently pair iex with:

Encoded commands (-EncodedCommand)

Web-based delivery (iex (New-Object Net.WebClient).DownloadString(...))

Living-off-the-land binaries (LOLBins)

This technique is used to:

Evade static detection rules

Circumvent input filters or script monitoring

Launch second-stage payloads after reconnaissance

**Detection Logic Recommendations**

To detect obfuscated or suspicious PowerShell activity:

Splunk query prototype:


index=main sourcetype=WinEventLog:Security EventCode=4688
(Process_Command_Line="*iex*" OR Process_Command_Line="*EncodedCommand*" OR Process_Command_Line="*DownloadString*")

Enhance detection by correlating:

Parent process: powershell.exe

Flags: -ExecutionPolicy Bypass, -NoProfile

Use of string construction, piping (|), or suspicious variable assignments

✅ Conclusion
This detection signal showcases a critical behavior pattern — obfuscated PowerShell execution. Even if the payload is harmless (e.g., calc), the execution style mimics real-world malware stages. Capturing and interpreting this signal helps train both detection logic and human triage intuition.

**Detection Signal 3A.3 – Scheduled Task Persistence Attempt**

User Action:
At 10:45 AM on July 1, a scheduled task named Updater was created using the following PowerShell command:

schtasks /create /tn "Updater" /tr "calc.exe" /sc onlogon

The goal was to simulate attacker persistence by configuring code (calc.exe) to launch on every user logon — a technique frequently used to maintain access after compromise.

Expected Log Artifacts:

Event ID 4698 (Task Created) in the Security log, indicating creation of a new scheduled task.

Event ID 4702 (Task Updated) if the task was modified.

Execution artifacts from calc.exe on subsequent logons (Event ID 4688: process creation).

Task record in Task Scheduler XML under C:\Windows\System32\Tasks.

**Observed Outcome:**

Task was successfully created and executed once during testing (visible calc.exe launch).

However, Event ID 4698 never appeared in Splunk, likely due to:

Audit policy misconfiguration at the time of task creation.

Delayed ingestion or indexing gaps in Splunk.

On subsequent logon (e.g., morning of July 2), the task did not re-trigger, possibly due to:

Windows logon behavior differences (sleep vs. restart vs. cold boot).

Splunk’s limited visibility due to license or permission limitations.

**Verification:**

A schtasks /query confirmed the scheduled task exists and is set to run at logon:

Status: Ready

Last Run Time: 7/1/25 3:14:50 PM

Last Result: 0 (Success)

Task also verified in Task Scheduler GUI under the name Updater.

**Detection Value (Conceptual):**

Event ID 4698 is a high-fidelity signal for persistence.

Malicious scheduled tasks are often used by:

Insider threats maintaining long-term access

Malware ensuring re-execution after reboot

Post-exploitation frameworks (e.g., Cobalt Strike, Empire)

Key fields to monitor include:

Task name (e.g., Updater, AdobeUpdate, etc.)

Task user context (non-admin, SYSTEM, NT AUTHORITY)

Task To Run field containing suspicious binaries or PowerShell

**Lessons Learned:**

Audit policies must be correctly configured before telemetry can be captured.

Manual validation through schtasks and Task Scheduler is crucial when log artifacts are missing.

Even failed Splunk visibility can be turned into an important learning moment about OS behavior and detection dependencies.



**3A.4 – Lateral Movement Simulation (RDP and SMB)**

Simulated Attacker Behavior:
At 11:41 AM on July 1, 2025, the attacker account attempted lateral movement to an internal host (192.168.56.102) using two classic administrative tools:

Remote Desktop Protocol (RDP):
Command executed: mstsc /v:192.168.56.102

Server Message Block (SMB):
Command executed: net use \\192.168.56.102\C$

Both actions were launched from PowerShell and occurred within the same 60-second window.

Detection Evidence in Splunk (Event ID 4688):

✅ mstsc.exe execution:

New Process Name: C:\Windows\System32\mstsc.exe

Process Command Line: "C:\WINDOWS\system32\mstsc.exe" /v:192.168.56.102

Creator Process: powershell.exe

Timestamp: 11:41:02 AM

✅ net.exe execution:

New Process Name: C:\Windows\System32\net.exe

Process Command Line: "C:\WINDOWS\system32\net.exe" use \\192.168.56.102\C$

Creator Process: powershell.exe

Timestamp: 11:41:27 AM

Both logs showed TokenElevationTypeLimited, consistent with a standard user launching privileged utilities.

**Detection Evidence in Wireshark (Correlated):**

TCP SYN packet sent to port 3389 (RDP)

TCP SYN packet sent to port 445 (SMB)

Destination: 192.168.56.102

No replies (host does not exist), confirming the simulation's stealth nature

**Security Interpretation:**

Attempts to access remote desktop or admin shares on internal IPs are clear signs of lateral movement.

Execution of these tools from PowerShell further elevates risk.

Although the simulated target host was nonexistent, the sequence mimics real-world attacker pivoting behavior.

These behaviors are especially suspicious when no helpdesk tickets, IT tools, or authorized remote workflows are present.

**Detection Recommendations:**

Flag any PowerShell-launched executions of mstsc.exe or net.exe, particularly with internal IP arguments.

Correlate with Logon Type 10 (RemoteInteractive) or 3 (Network) to identify interactive or programmatic remote access.

Enrich detection with NetFlow or packet capture analysis where available.



**3A.5 – Privilege Escalation Behavior (VSSAdmin Enumeration)**

**Simulated Action**

At 11:58 AM on July 1, 2025, the attacker account executed:


vssadmin list shadows

This is a common enumeration command used to list Volume Shadow Copies on a Windows system. Adversaries may use this technique to identify backup copies they can delete to prevent system recovery or forensic rollback (e.g., in preparation for ransomware deployment).

**Detection Evidence (Splunk: EventCode 4688)**

Field	           Value
_time	           2025-07-01 11:58:10
Account_Name	   Steve
New_Process_Name   C:\Windows\System32\vssadmin.exe
Process_Command_Line	"C:\WINDOWS\system32\vssadmin.exe" list shadows
Creator_Process_Name	C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe
Security_ID	            S-1-5-21-2302811823-65636539-1088841326-1004
Logon_ID	            0x30EF5CDE

Interpretation
This is an unusual process command for standard users and typically indicates host-level reconnaissance or pre-ransomware staging.

Execution via PowerShell as the parent process adds further suspicion, consistent with living-off-the-land (LOTL) tactics.

While vssadmin may be used by backup or IT tools, in this simulation context it reflects adversarial behavior and should be flagged for alerting or deeper triage.



**Conclusion: Insider Threat Simulation – Final Observations**

This project simulated the behavior of a malicious insider on a Windows 11 host to generate detectable telemetry for use in SIEM-based detection and host triage. The exercise focused on five core behaviors that commonly indicate internal compromise or abuse:

Repeated failed logon attempts

Successful local logon

Persistence via scheduled task

Lateral movement attempts (RDP and SMB)

Privilege escalation behavior (vssadmin.exe)

Splunk successfully ingested Windows Security logs and captured key events across all five stages. Detection was confirmed through filtered Event ID 4688 entries and command line visibility. Wireshark was used in parallel to validate outbound connection attempts for lateral movement.

**Key Takeaways**

Telemetry Validation Is Non-Trivial: Significant time was spent enabling and validating audit policies, user permissions, and Splunk configuration to ensure full log visibility. These challenges reflect real-world deployment and monitoring issues.

Behavior-Based Detection Works: Even simple attacker simulations like launching mstsc.exe, net use, and vssadmin.exe triggered clear, filterable events when visibility was properly configured.

Repetition Builds Confidence: By recreating common attacker behaviors in a controlled environment and matching them to specific log entries, we built fluency in host telemetry interpretation — a core SOC skill.

**Next Steps**
Now that the environment is configured and core detection patterns are understood, a follow-up project will repeat this methodology with slight variations in behavior and process execution. The goal is to reinforce detection workflows while minimizing time spent on reconfiguration.

**System Configuration Status & Lessons Learned**

This project required extensive configuration work at the Windows OS and Splunk levels to enable full visibility into host activity. These changes were essential to support telemetry collection and SIEM-based analysis but introduced complexity worth documenting.

✅ Key Configuration Steps Completed
Audit Policy Changes

Enabled Audit Process Creation with command line logging (AuditProcessCreation) set to "Success and Failure"

Verified logs were being written by confirming Event ID 4688 entries with populated command lines

Splunk Permissions and Access

Granted the Splunk service account (“.\Steve”) the Log on as a service right via secpol.msc

Resolved permissions errors blocking access to the Security log by reconfiguring the Splunk input and restarting the service

Task Scheduler Validation

Created a scheduled task (Updater) to simulate persistence

Verified task creation via Event ID 4698, though visibility was inconsistent in Splunk despite successful execution

Command Prompt and PowerShell Use

Repeated use of administrator-level CMD and PowerShell was required for:

Task creation and inspection

Real-time testing of attacker behaviors

Updating audit policies and refreshing Group Policy with gpupdate /force

**Configuration Challenges Faced** 

Splunk License Limitation

Encountered license expiration message due to event volume during testing

Required restart and workaround to resume searching logs

Windows File Explorer Confusion

Difficulty accessing the C:\Windows\System32\Tasks directory via GUI due to File Explorer interface quirks and address bar issues

Event Visibility Gaps

Despite proper scheduled task creation and visible execution in Task Scheduler, corresponding 4698 logs were not always ingested by Splunk, possibly due to indexing delays or internal caching

Manual Restarts and Delayed Effects

Some system behaviors (e.g., scheduled task launch) did not trigger consistently after logoff or reboot, highlighting the unreliability of some persistence simulation techniques in test environments





